In [1]:
using CSV, DataFrames, Statistics, HTTP

In [5]:
url = "https://raw.githubusercontent.com/ArieBeresteanu/Bounds.jl/main/Fortran/Chtdata.csv"

http_response = HTTP.get(url)
file = CSV.File(http_response.body,header=false)

cht = DataFrame(file)

rename!(cht, :Column1 => :yl, :Column2 => :yu, :Column3 => :edu)
first(cht,3)

,yl,yu,edu
,Float64,Float64,Int64
1,4.0,4.0969,28
2,4.5441,4.699,28
3,4.1761,4.301,34


In [9]:
s = describe(cht, :mean, cols=[:yl,:yu])
H0 = [s[1,2] s[2,2]]

1×2 Matrix{Float64}:
 4.43466  4.96743

In [4]:
include("")